### Загружаем базовую модель

In [1]:
from vlm_finetune import AutoVlmModel, ImageProcessor


model = AutoVlmModel.from_name(
    model_name="qwen", 
    model_path="Qwen/Qwen2.5-VL-7B-Instruct",
    image_processor=ImageProcessor(image_size=(336, 336)) # ресайзим картинки, чтобы сэкономить память
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

### Тестим на нашем датасете, для примера возьмём рандомный сэмпл

In [2]:
import json

with open("data/llava_vet_dataset_test.json", "r") as file:
    sample = json.loads(file.read())[0]

In [3]:
sample

{'image_path': 'data/грудная/52а.png',
 'answer': 'легкие расправлены без очаговых и инфильтрированных теней. легочной рисунок не изменен. тень сердца и аорты без изменений. просвет трахеи не нарушен. уплотнение хрящей гортани, увеличение в объеме.  на момент проведения исследования выявлены признаки ларингита'}

In [4]:
prompt = "Поставь вероятный диагноз по рентгенограмме."

In [5]:
%%time

model_answer = model.predict(image=sample["image_path"], prompt=prompt)

CPU times: user 17.7 s, sys: 4.44 s, total: 22.2 s
Wall time: 22.8 s


In [6]:
model_answer

'На основе предоставленного рентгеновского снимка, можно предположить следующие возможные диагнозы:\n\n1. **Остеохондропатия**: Это может быть обусловлено нарушением развития костей, что приводит к деформации и болезненности.\n\n2. **Остеохондроз**: Это заболевание характеризуется уменьшением толщины хрящевой ткани между позвонками, что может привести к боли и ограничению движений.\n\n3. **Остеохондритид**: Это воспаление хрящевой ткани, которое может быть вызвано травмой или генетическими факторами.\n\n4. **Остеохондрома**: Это образование в виде узла, которое может быть доброкачественным или злокачественным.\n\n5. **Остеохондрогенезис**: Это нарушение процесса образования костей, что может привести к деформациям и болезненности.\n\nДля точного диагноза необходимо обратиться к специ'

In [7]:
import evaluate


bertscore = evaluate.load("bertscore")

In [8]:
bertscore.compute(predictions=[model_answer], references=[sample["answer"]], lang="ru")

{'precision': [0.5791910886764526],
 'recall': [0.6415655612945557],
 'f1': [0.6087848544120789],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.57.1)'}

**Модель ответила абсолютно бессвязно, просто повторила наш вопрос**

### Зафайнтюним базовую модель с помощью фреймворка
Параметры берём дефолтные (они уже зашиты во фреймворк), параметры можно менять и "докручивать"

In [9]:
model.finetune(
    dataset_path="data/llava_vet_dataset_train.json", 
    output_dir="models/qwen_tuned", 
    prompt=prompt, 
    num_train_epochs=3
)

2025-10-29 19:05:13 [INFO] vlm_finetune.base.model: Используется устройство: cuda:0
2025-10-29 19:05:13 [INFO] vlm_finetune.base.model: Количество обучаемых параметров: 10092544
2025-10-29 19:05:13 [INFO] vlm_finetune.base.model: Начинаем finetune...


trainable params: 10,092,544 || all params: 8,302,259,200 || trainable%: 0.1216


Step,Training Loss
10,2.589500
20,1.844900


2025-10-29 20:02:06 [INFO] vlm_finetune.base.model: Модель успешно затюнена и сохранена в models/qwen_tuned
2025-10-29 20:02:06 [INFO] vlm_finetune.base.model: !Для предсказаний будет использоваться доученная модель!


### Смотрим тот же сэмпл из нашего датасета

In [10]:
%%time

model_answer = model.predict(image=sample["image_path"], prompt=prompt)

CPU times: user 9.65 s, sys: 1.97 s, total: 11.6 s
Wall time: 11.8 s


In [11]:
model_answer

'Рентгенограмма животного показывает нормальное расположение костей скелета, без признаков переломов или деформаций. Однако, для точного диагноза необходимо учитывать клиническую картину и результаты других методов исследования.'

In [12]:
bertscore.compute(predictions=[model_answer], references=[sample["answer"]], lang="ru")

{'precision': [0.6712049841880798],
 'recall': [0.6571913957595825],
 'f1': [0.6641243100166321],
 'hashcode': 'bert-base-multilingual-cased_L9_no-idf_version=0.3.12(hug_trans=4.57.1)'}


**Модель отвечает по теме**